# テーブルの結合 (groupby, agg, merge)

In [ ]:
!git clone https://github.com/ironerumi/fe_workshop.git -q

In [ ]:
import numpy as np
import scipy as sp
from scipy import optimize
import pandas as pd
from pandas import DataFrame, Series

pd.set_option("display.max_columns", 100)

from google.colab import files

In [ ]:
# collaboratoryにLendingClub50000.csv, spx2.csvをアップロードする。
# uploaded = files.upload()

In [ ]:
# ファイルを読み込む
df = pd.read_csv("fe_workshop/dataset/LendingClub50000.csv")
spx = pd.read_csv("fe_workshop/dataset/spx2.csv", parse_dates=["date"])

In [12]:
df.head().T

,0,1,2,3,4
申込ID,1529851,1824764,403548,646411,552526
メンバーID,1793711,2126933,442721,799671,711946
年,2012,2012,2009,2011,2010
月,9,11,5,1,7
ローン申請額,16000,3600,19200,21000,4000
借り入れ目的（大分類）,debt_consolidation,debt_consolidation,wedding,credit_card,other
借り入れ目的（小分類）,Debt Consolidation / Final Wedding Exp,Crush Credit Cards,Consolidate debt and pay for wedding,Wells BofA Credit Card Refi,Debt Consolidation
勤務先,PwC,Morgan Stanley Smith Barney,Aggregate Knowledge,Emerson Process Management,Home Depot
勤続年数,3,2,1,1,1
居住形態,RENT,RENT,RENT,RENT,RENT


In [13]:
print(spx.head(10).to_markdown())

|    | date                |   close |
|---:|:--------------------|--------:|
|  0 | 1986-01-02 00:00:00 |  209.59 |
|  1 | 1986-01-03 00:00:00 |  210.88 |
|  2 | 1986-01-06 00:00:00 |  210.65 |
|  3 | 1986-01-07 00:00:00 |  213.8  |
|  4 | 1986-01-08 00:00:00 |  207.97 |
|  5 | 1986-01-09 00:00:00 |  206.11 |
|  6 | 1986-01-10 00:00:00 |  205.96 |
|  7 | 1986-01-13 00:00:00 |  206.72 |
|  8 | 1986-01-14 00:00:00 |  206.64 |
|  9 | 1986-01-15 00:00:00 |  208.26 |


In [14]:
# 年と月を日付から抽出する
spx["年"] = spx.date.dt.year
spx["月"] = spx.date.dt.month
print(spx.head(10).to_markdown())

|    | date                |   close |   年 |   月 |
|---:|:--------------------|--------:|-----:|-----:|
|  0 | 1986-01-02 00:00:00 |  209.59 | 1986 |    1 |
|  1 | 1986-01-03 00:00:00 |  210.88 | 1986 |    1 |
|  2 | 1986-01-06 00:00:00 |  210.65 | 1986 |    1 |
|  3 | 1986-01-07 00:00:00 |  213.8  | 1986 |    1 |
|  4 | 1986-01-08 00:00:00 |  207.97 | 1986 |    1 |
|  5 | 1986-01-09 00:00:00 |  206.11 | 1986 |    1 |
|  6 | 1986-01-10 00:00:00 |  205.96 | 1986 |    1 |
|  7 | 1986-01-13 00:00:00 |  206.72 | 1986 |    1 |
|  8 | 1986-01-14 00:00:00 |  206.64 | 1986 |    1 |
|  9 | 1986-01-15 00:00:00 |  208.26 | 1986 |    1 |


In [15]:
# 年と月でgroupbyしてそれぞれS&P終値の統計値を算出する
summary = spx.groupby(["年", "月"]).agg(["mean", "min", "max"])
summary.columns = ["_".join(x) for x in summary.columns.ravel()]
summary.reset_index(inplace=True)
print(summary.head().to_markdown())

|    |   年 |   月 |   close_mean |   close_min |   close_max |
|---:|-----:|-----:|-------------:|------------:|------------:|
|  0 | 1986 |    1 |      208.194 |      203.49 |      213.8  |
|  1 | 1986 |    2 |      219.365 |      212.79 |      226.92 |
|  2 | 1986 |    3 |      232.327 |      224.34 |      238.97 |
|  3 | 1986 |    4 |      237.968 |      228.63 |      244.74 |
|  4 | 1986 |    5 |      238.459 |      232.76 |      247.98 |


In [16]:
# ローンデータとマージする
df = df.merge(summary, on=["年", "月"], how="left")

In [17]:
df.head().T

,0,1,2,3,4
申込ID,1529851,1824764,403548,646411,552526
メンバーID,1793711,2126933,442721,799671,711946
年,2012,2012,2009,2011,2010
月,9,11,5,1,7
ローン申請額,16000,3600,19200,21000,4000
借り入れ目的（大分類）,debt_consolidation,debt_consolidation,wedding,credit_card,other
借り入れ目的（小分類）,Debt Consolidation / Final Wedding Exp,Crush Credit Cards,Consolidate debt and pay for wedding,Wells BofA Credit Card Refi,Debt Consolidation
勤務先,PwC,Morgan Stanley Smith Barney,Aggregate Knowledge,Emerson Process Management,Home Depot
勤続年数,3,2,1,1,1
居住形態,RENT,RENT,RENT,RENT,RENT


In [ ]:
df.to_csv("LendingClub50000_spx.csv", index=False)

In [ ]:
# ダウンロードしたら、DataRobotでもう一度予測してみよう
files.download("LendingClub50000_spx.csv")